In [18]:
using DataFrames, CSV
using JLD
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions
using XLSX

In [2]:
liverpool_data = CSV.read("Liverpool_FIFA_22.csv", DataFrame)
n_players = nrow(liverpool_data)

33

In [3]:
all_teams = CSV.read("Team_Ratings.csv", DataFrame)
n_positions = 27
C = CSV.read("fixture_list.csv", DataFrame)
n_games = nrow(C)

38

In [4]:
GK = "gk"
defenders = ["lwb", "lb", "lcb", "cb", "rcb", "rb", "rwb"]
midfielders = ["lam", "cam", "ram", "lm", "lcm", "cm", "rcm", "rm", "ldm", "cdm", "rdm"]
forwards = ["ls", "st", "rs", "lw", "lf", "cf", "rf", "rw"]
center_backs = ["lcb", "cb", "rcb"]
right_backs = ["rwb", "rb"]
left_backs = ["lwb", "lb"]
cdms = ["ldm", "cdm", "rdm"]
cms = ["lcm", "cm", "rcm"]
cams = ["lam", "cam", "ram"]
left_wingers = ["lw", "lm"]
right_wingers = ["rw", "rm"]
strikers = ["ls", "st", "rs", "lf", "cf", "rf"]

6-element Vector{String}:
 "ls"
 "st"
 "rs"
 "lf"
 "cf"
 "rf"

In [5]:
player_pos = ["ls", "st", "rs", "lw", "lf", "cf", "rf", "rw", "lam", "cam", "ram", "lm", "lcm", "cm", "rcm", "rm", "lwb", "ldm", "cdm", "rdm", "rwb", "lb", "lcb", "cb", "rcb", "rb", "gk"]
pos_to_ind = Dict()
ind_to_pos = Dict()
index = 1
for pos in player_pos
    pos_to_ind[pos] = index
    ind_to_pos[index] = pos
    index += 1
end
pos_to_ind["st"]

2

In [6]:
player_pos[27]

"gk"

In [7]:
defensive_pos = Set(["lb", "lcb", "cb", "rcb", "rb", "ldm", "cdm", "rdm"])
central_pos = Set(["ldm", "cdm", "rdm", "lcm", "cm", "rcm", "lam", "cam", "ram"])
forward_pos = Set(forwards)
wing_pos = Set(["lwb", "rwb", "lm", "rm", "lw", "rw"])
wingback_pos = Set(["lwb", "rwb", "lb", "rb"])
union(wingback_pos, wing_pos)

Set{String} with 8 elements:
  "rm"
  "rwb"
  "lb"
  "rw"
  "lw"
  "lm"
  "lwb"
  "rb"

In [8]:
function acceptable_positions(player_orig)
    acceptable_pos = Set()
    for k in player_orig
        push!(acceptable_pos, lowercase(k))
    end
    return acceptable_pos
end

R = zeros(Int64, (n_players, n_positions))
for i in 1:n_players
    player_orig = split(liverpool_data[i, "player_positions"], ", ")
    acceptable_pos = acceptable_positions(player_orig)
    for pos in acceptable_pos
        R[i, pos_to_ind[pos]] = liverpool_data[i, pos]
    end
end
R

33×27 Matrix{Int64}:
 0   0  0   0  0   0  0   0  0  0  0  0  …  0   0  0  0   0  0  89  0   0   0
 0   0  0  87  0   0  0   0  0  0  0  0     0   0  0  0   0  0   0  0   0   0
 0   0  0   0  0   0  0  88  0  0  0  0     0   0  0  0   0  0   0  0   0   0
 0   0  0   0  0   0  0   0  0  0  0  0     0   0  0  0   0  0   0  0   0  90
 0   0  0   0  0   0  0   0  0  0  0  0     0   0  0  0  87  0   0  0   0   0
 0   0  0   0  0   0  0   0  0  0  0  0  …  0   0  0  0   0  0   0  0  86   0
 0   0  0   0  0   0  0   0  0  0  0  0     0  83  0  0   0  0   0  0   0   0
 0   0  0   0  0   0  0   0  0  0  0  0     0  87  0  0   0  0  87  0   0   0
 0   0  0   0  0  83  0   0  0  0  0  0     0   0  0  0   0  0   0  0   0   0
 0   0  0   0  0   0  0   0  0  0  0  0     0  84  0  0   0  0   0  0   0   0
 0   0  0   0  0   0  0   0  0  0  0  0  …  0   0  0  0   0  0  83  0   0   0
 0  84  0  82  0   0  0   0  0  0  0  0     0   0  0  0   0  0   0  0   0   0
 0   0  0   0  0   0  0   0  0  0  0  0    

In [9]:
pos_discount = 0.99
function penalty(player_number, game_number, x)
    if game_number == 1
        return 1
    end
    if sum(x[player_number, game_number-1, k] for k in 1:n_positions) == 0
        return 1
    end
    max_consecutive = 0
    for j in game_number-1:-1:1
        if sum(x[player_number, j, k] for k in 1:n_positions) == 0
            break
        else
            max_consecutive += 1
        end
    end
    return pos_discount^max_consecutive
end

penalty (generic function with 1 method)

In [10]:
# model = Model(with_optimizer(Gurobi.Optimizer))
model = Model(Gurobi.Optimizer)

### Decision Variables ###

### x[i][j][k]. player i, game j, position k ###
@variable(model, x[i=1:n_players, j=1:n_games, k=1:n_positions], Bin)

### z variable, for whether we go for win or not ###
@variable(model, z[j = 1:n_games], Bin)


### Match Winning Constraint ###
### R is player ratings, R[i][k] is player i for position k ###
### C is the opposing team rating, C[j] is the rating of the opposing team for game j ###

for j in 1:n_games
    @constraint(model, sum(sum(R[i, k]*x[i, j, k] for i in 1:n_players) for k in 1:n_positions)/11 >= C[j, 2]*z[j])
end

### Player Constraints ###

### Each player can only play at most once in a game ###
for j in 1:n_games
    for i in 1:n_players
        @constraint(model, sum(x[i, j, k] for k in 1:n_positions) <= 1)
    end
end

### Each player can play a maximum number of games ###
for i in 1:n_players
    @constraint(model, sum(sum(x[i, j, k] for j in 1:n_games) for k in 1:n_positions) <= liverpool_data[i, "max_games"])
end

### Can only have 11 players in game ###
for j in 1:n_games
    @constraint(model, sum(sum(x[i, j, k] for k in 1:n_positions) for i in 1:n_players) == 11)

    ### 1 GK per game ###
    @constraint(model, sum(x[i, j, pos_to_ind[GK]] for i in 1:n_players) == 1)

    ### 4 Defenders per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in defenders) == 4)

    ### 3 - 5 Midfielders per game ### 
    @constraint(model, 3 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in midfielders) <= 5)

    ### 1 - 3 Forwards per game ###
    @constraint(model, 1 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in forwards) <= 3)

    # ### 2 CBs per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in center_backs) == 2)

    # ### 1 RB per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in right_backs) == 1)

    # ### 1 LB per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in left_backs) == 1)

    # ### 0 - 2 CDMs per game ###
    @constraint(model, 0 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in cdms) <= 2)

    # ### 0 - 3 CMs per game ###
    @constraint(model, 0 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in cms) <= 3)

    # ### 0 - 3 CAMS per game ###
    @constraint(model, 0 <= sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in cams) <= 3)

    # ### 0 - 1 LW, LM per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in left_wingers) <= 1)

    # ### 0 - 1 RW, RM per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in right_wingers) <= 1)

    # ### 1 ST per game ###
    @constraint(model, sum(sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) for k in strikers) == 1)
end

### Objective Function ###
@objective(model, Max, sum(z[j] for j in 1:n_games))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-28


z[1] + z[2] + z[3] + z[4] + z[5] + z[6] + z[7] + z[8] + z[9] + z[10] + z[11] + z[12] + z[13] + z[14] + z[15] + z[16] + z[17] + z[18] + z[19] + z[20] + z[21] + z[22] + z[23] + z[24] + z[25] + z[26] + z[27] + z[28] + z[29] + z[30] + z[31] + z[32] + z[33] + z[34] + z[35] + z[36] + z[37] + z[38]

In [11]:
optimize!(model)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1857 rows, 34086 columns and 169974 nonzeros
Model fingerprint: 0x47fc172a
Variable types: 190 continuous, 33896 integer (33896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 5e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 76 rows and 18962 columns
Presolve time: 0.13s
Presolved: 1781 rows, 15124 columns, 55708 nonzeros
Variable types: 0 continuous, 15124 integer (15086 binary)

Root relaxation: objective 3.800000e+01, 729 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                      38.0000000   38.00000  0.00%     -    0s
     0     0   38.00000    0   97   38.00000   38.00000  0

In [12]:
opt_x = value.(x)
max_rating = 0
index = 0
for j in 1:n_games
    avg_rating = sum(sum(R[i, k]*opt_x[i, j, k] for i in 1:n_players) for k in 1:n_positions)/11
    if avg_rating > max_rating
        max_rating = avg_rating
        index = j
    end
end
println(index, " ", max_rating)

7 85.27272727272727


In [13]:
function matrix_to_lineup(x, j)
    player_to_pos = Dict()
    for i in 1:n_players
        for k in 1:n_positions
            cur_val = x[i, j, k]
            if cur_val == 1
                player_name = liverpool_data[i, "short_name"]
                player_pos = ind_to_pos[k]
                player_to_pos[player_name] = player_pos
            end
        end
    end
    return player_to_pos
end

println(matrix_to_lineup(opt_x, index))

Dict{Any, Any}(String31("M. Salah") => "rw", String31("N. Keïta") => "cm", String31("V. van Dijk") => "cb", String31("J. Henderson") => "cm", String31("D. Origi") => "st", String31("T. Alexander-Arnold") => "rb", String31("Fabinho") => "cb", String31("A. Robertson") => "lb", String31("Alisson") => "gk", String31("Thiago") => "cm", String31("Diogo Jota") => "lw")


In [14]:
function every_lineup(x)
    game_to_lineup = Dict()
    for j in 1:n_games
        game_to_lineup[j] = matrix_to_lineup(x, j)
    end
    return game_to_lineup
end

line_ups = every_lineup(opt_x)

Dict{Any, Any} with 38 entries:
  5  => Dict{Any, Any}(String31("J. Matip")=>"cb", String31("J. Milner")=>"lb",…
  16 => Dict{Any, Any}(String31("J. Milner")=>"rb", String31("C. Jones")=>"cm",…
  20 => Dict{Any, Any}(String31("J. Milner")=>"lb", String31("T. Minamino")=>"l…
  35 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Milner")=>"rb", …
  12 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Milner")=>"rb", …
  24 => Dict{Any, Any}(String31("M. Salah")=>"rw", String31("V. van Dijk")=>"cb…
  28 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Milner")=>"rb", …
  8  => Dict{Any, Any}(String31("J. Matip")=>"cb", String31("T. Minamino")=>"st…
  17 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Milner")=>"cm", …
  30 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("J. Milner")=>"cm", …
  1  => Dict{Any, Any}(String31("T. Minamino")=>"st", String31("C. Jones")=>"cm…
  19 => Dict{Any, Any}(String31("S. Mané")=>"lw", String31("K. Tsimikas")=>"l

In [15]:
function dict_to_df(lineup_dict)
    player_names, player_pos = [], []
    for (key, value) in lineup_dict
        push!(player_names, key)
        push!(player_pos, value)
    end
    df = DataFrame(player_name = player_names, position = player_pos)
    return df
end
dict_to_df(line_ups[1])

Row,player_name,position
,Any,Any
1,T. Minamino,st
2,C. Jones,cm
3,M. Salah,rw
4,N. Keïta,cm
5,A. Oxlade-Chamberlain,lw
6,N. Williams,rb
7,A. Robertson,lb
8,Alisson,gk
9,Thiago,cm


In [16]:
for game in 1:n_games
    cur_lineup = line_ups[game]
    df = dict_to_df(cur_lineup)
    CSV.write("lineups\\lineup_game_$game.csv", df)
end

In [23]:
fixtures = C[!, "Team"]
XLSX.openxlsx("lineups.xlsx", mode="w") do xf
    XLSX.rename!(xf[1], fixtures[1] * "_1")
    teams = Set([fixtures[1]])
    sheet = xf[fixtures[1] * "_1"]
    cur_lineup = line_ups[1]
    df = dict_to_df(cur_lineup)
    for r in 1:size(df,1), c in 1:size(df,2)
        sheet[XLSX.CellRef(r , c )] = df[r,c]
    end
    for game in 2:n_games
        cur_team = fixtures[game]
        if cur_team in teams
            sheet_name = cur_team * "_2"
        else
            sheet_name = cur_team * "_1"
            push!(teams, cur_team)
        end
        XLSX.addsheet!(xf, sheet_name)
        sheet = xf[sheet_name]
        cur_lineup = line_ups[game]
        df = dict_to_df(cur_lineup)
        for r in 1:size(df,1), c in 1:size(df,2)
            sheet[XLSX.CellRef(r , c )] = df[r,c]
        end
    end
end